#Setup

In [1]:
!pip install -q

ERROR: You must give at least one requirement to install (see "pip help install")


In [2]:
!pip install -q SpeechRecognition pydub gradio PyPDF2

In [3]:
import os
from google.colab import drive

import torch
import warnings

from transformers import pipeline
import PyPDF2
import re

import speech_recognition as sr
from pydub import AudioSegment
from pydub.silence import split_on_silence
from concurrent.futures import ThreadPoolExecutor

import gradio as gr
from pathlib import Path
import numpy as np

In [4]:
drive.mount('/content/drive')
path = 'GDSC AI HACK - Hackattoni'

os.chdir(f'/content/drive/MyDrive/{path}')
os.getcwd()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


'/content/drive/.shortcut-targets-by-id/1kBsikYvE9HO3QWRWOQCMZ3wDGrZxlstE/GDSC AI HACK - Hackattoni'

#PDF-to-TXT

In [5]:
def split_pdf_phrases(pdf_path):
    # Open the PDF file
    with open(pdf_path, 'rb') as file:
        # Create a PDF file reader object
        reader = PyPDF2.PdfReader(file)

        # Initialize an empty list to store phrases
        phrases = []

        # Iterate through each page in the PDF
        for page_num in range(len(reader.pages)):
            # Extract text from the page
            page_text = reader.pages[page_num].extract_text()

            # Split text into phrases based on punctuation marks or line breaks
            # You can adjust this regular expression to fit your specific needs
            page_phrases = re.split(r'[.!?]\s*', page_text)

            # Remove empty phrases
            page_phrases = [phrase.strip().replace('\n','') for phrase in page_phrases if phrase.strip()]

            # Add page phrases to the list of all phrases
            phrases.extend(page_phrases)

    return phrases

#Speech-to-TXT

In [10]:
r = sr.Recognizer()

def transcribe_audio(chunk):
    text = ""
    audio = sr.AudioData(chunk.raw_data, chunk.frame_rate, 2)
    text = r.recognize_google(audio)
    return text

def process_chunk(chunk, index):
    try:
        text = transcribe_audio(chunk)
    except sr.UnknownValueError as e:
        return ""
    else:
        text = f"{text.capitalize()}. "
        return text

def get_large_audio_transcription_on_silence(path):
    sound = AudioSegment.from_file(path)
    chunks = split_on_silence(sound,
                              min_silence_len=500,
                              silence_thresh=sound.dBFS-14,
                              keep_silence=0)

    folder_name = "audio-chunks"
    if not os.path.isdir(folder_name):
        os.mkdir(folder_name)

    whole_text = []
    with ThreadPoolExecutor() as executor:
        futures = []
        for i, audio_chunk in enumerate(chunks, start=1):
            chunk_filename = os.path.join(folder_name, f"chunk{i}.wav")
            audio_chunk.export(chunk_filename, format="wav")
            futures.append(executor.submit(process_chunk, audio_chunk, i))

        for future in futures:
            if future.result() != '':
              whole_text.append(future.result())

    return whole_text

#Fact-checking

In [7]:
# Set the device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load a pretrained model suitable for text classification
classifier = pipeline("text-classification", model="roberta-large-mnli", device=device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of the model checkpoint at roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializi

In [8]:
def fact_check(speech_txt, pdf_txt):
  outputs=[]
  for s in speech_txt:
    for p in pdf_txt:
      input_text = f"{p} {s}"
      result = classifier(input_text)
      if result[0]['label'] == 'CONTRADICTION' :
        outputs.append((p,s))
  return outputs

#Front-END

In [11]:
def process_pdf(pdf_file):
    text = split_pdf_phrases(pdf_file)
    return text

def process_audio(audio_file):
    # Process the audio
    speech_txt = get_large_audio_transcription_on_silence(audio_file)

    # Return the result
    return speech_txt

def process(pdf_input, audio_input):
    pdf_txt = process_pdf(pdf_input)
    speech_txt = process_audio(audio_input)

    outputs = fact_check(speech_txt, pdf_txt)
    real_out = ""
    for c in outputs:
      real_out += f'Your sentence: "{c[0]}" is in contradiction with the following sentence "{c[1]}"\n'
    return pdf_txt, speech_txt, real_out

with gr.Blocks() as demo:

    gr.Markdown()

    with gr.Row():
        with gr.Column():
            pdf_inp = gr.File(scale=1,file_types=['.pdf'])
            pdf_name = gr.Textbox(label="File uploaded", visible=False)

        with gr.Column():
            audio_inp = gr.Audio(scale=1, type="filepath")

    with gr.Row():
        with gr.Column():
            out_pdf = gr.Textbox("", label='Pdf text')

        with gr.Column():
            out_speech = gr.Textbox("", label='Speech text')

    with gr.Row():
        btn = gr.Button("Elaborate inputs")
    with gr.Row():
        out = gr.Textbox("output", label='')

    btn.click(fn=process, inputs=[pdf_inp,audio_inp], outputs=[out_pdf,out_speech, out])

demo.launch()



Thanks for being a Gradio user! If you have questions or feedback, please join our Discord server and chat with us: https://discord.gg/feTf9x3ZSB
Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://e66edd90b5df87ad32.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
